<a href="https://colab.research.google.com/github/lkkcpaul/BostonHousePrice/blob/main/BostonHousePriceRandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
# Comment this if the data visualisations doesn't work on your side
%matplotlib inline

In [2]:
train_file_url = "https://raw.githubusercontent.com/lkkcpaul/BostonHousePrice/main/train.csv"
dataset_df = pd.read_csv(train_file_url)
print("Full train dataset shape is {}".format(dataset_df.shape))

Full train dataset shape is (1460, 81)


In [3]:
dataset_df.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
dataset_df.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [5]:
dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

What to do with null objects? There are a few with many nulls.

Alley: NA in alley means no alley access, which contains information

FireplaceQu: NA means no fireplace, which is encoded in "Fireplaces"

PoolQC: NA means no pool. That only 7 houses have pools, we probably don't need to distinguish between the quality of the pools

Fence: NA means no fence

MiscFeature: 49 of them are Shed. So we probably just make a column of "Shed" and delete this one

In [6]:
# Find the numeric columns that have missing values
# Decide how to fill them
has_numeric_na = dataset_df.select_dtypes(include=np.number).isna().any()
has_numeric_na[has_numeric_na]

LotFrontage    True
MasVnrArea     True
GarageYrBlt    True
dtype: bool

In [ ]:
# We will impute Lot Frontage with mean,
# MasVnrArea with 0
# And GarageYrBlt with majority (since missing values just mean there's no garage)

In [14]:
drop_col = ['Id','PoolArea', 'PoolQC']
fillna_dict = {'MasVnrArea':0, 
               'LotFrontage':dataset_df.LotFrontage.mean(),
               'GarageYrBlt':dataset_df.GarageYrBlt.mean()
             }

def preprocess(data):
  temp = data.drop(drop_col, axis=1)
  temp['Shed']=0
  temp['Shed'][temp['MiscFeature']=='Shed'] = 1
  temp.drop('MiscFeature',axis=1,inplace=True)
  temp.fillna(fillna_dict,inplace=True)
  return temp

In [15]:
df = preprocess(dataset_df)

<ipython-input-14-e924caf0fb0a>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['Shed'][temp['MiscFeature']=='Shed'] = 1


In [53]:
from sklearn.preprocessing import OneHotEncoder
X_w_cat = df.drop('SalePrice',axis=1)
X_w_cat['MSSubClass'] = X_w_cat['MSSubClass'].map(str)
y = df.SalePrice
OHE = OneHotEncoder(handle_unknown = 'ignore',sparse_output=False)
X = pd.DataFrame(OHE.fit_transform(X_w_cat))

In [54]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)

In [60]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
rf = RandomForestRegressor(300)
rf.fit(X_train,y_train)
yhat = rf.predict(X_test)
print("The mse using random forest is", mean_squared_error(y_test,yhat))

The mse using random forest is 1172063958.5047193


In [61]:
import math
math.sqrt(mean_squared_error(y_test,yhat))

34235.41964843895

In [62]:
rf100 = RandomForestRegressor(100)
rf100.fit(X_train,y_train)
yhat = rf.predict(X_test)
print("The rmse using random forest with 100 trees is", math.sqrt(mean_squared_error(y_test,yhat)))

The rmse using random forest with 100 trees is 34235.41964843895
